In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from numpy import cov
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
df=pd.read_csv('../input/africa-economic-banking-and-systemic-crisis-data/african_crises.csv')

# Quick overview and data preparation

In [ ]:
df.info()

In [ ]:
df.tail(10)

In [ ]:
df.describe()


We need to get rid of value 2 in currency_crises as there shoul be only 0 and 1.

In [ ]:
df['currency_crises'] = df['currency_crises'].replace(2,np.nan)
df = df.dropna()

In [ ]:
years = df.year.unique()
len(years)

Totally we have 155 years in the dataset

In [ ]:
countries = df["country"].unique()
countries


In [ ]:
len(countries)

And 13 African countries

In [ ]:
df["banking_crises"] = np.where(df['banking_crisis'] == "crisis", 1, 0)

We need a proper column with 0/1 values instead of str.

Okay, let's start. The idea is to identify what are the interconnections in the dataset, what factors lead to crises in Africa and what does not. It's also good to know whether being independent affected financial stability of the African countries.

In [ ]:
dependent = df[df.independence == 0]
independent =  df[df.independence == 1]

# Systemic crisis

Systemic crisis means the nationwide banking crisis which its impact spreads to the whole banking sector. There are two conditions to be met: 
1) significant signs of financial distress are in a majority part of the banking system
2) significant banking policy intervention measures are implemented by the appropriate financial authorities in response to significant losses amongst the individual components of the banking system
In this work we're going to consider systemic crisis applied to the economy of particular country.
Obviously, the more often such crises happen, the worse economic situation in country is. 

[IGI Global](https://www.igi-global.com/dictionary/predicting-global-financial-meltdown-and-systemic-banking-failure/51065)

In [ ]:
sys_crises_per_country_dependent = dependent.groupby("country",as_index=False)["systemic_crisis"].sum() 
sys_crises_per_country_independent = independent.groupby("country",as_index=False)["systemic_crisis"].sum() 
countrues_sorted_dep = sys_crises_per_country_dependent.sort_values(by = ["systemic_crisis"], ascending=False)
countrues_sorted_indep = sys_crises_per_country_independent.sort_values(by = ["systemic_crisis"], ascending=False)

In [ ]:
plt.figure(figsize=(15,7))
plt.bar(countrues_sorted_dep["country"], countrues_sorted_dep["systemic_crisis"],color=(0.7, 0.7, 0.7, 0.6))
plt.title("Systemic crises for countries while being dependent")
plt.show()
plt.figure(figsize=(15,7))
plt.bar(countrues_sorted_indep["country"], countrues_sorted_indep["systemic_crisis"],color=(0.3, 0.4, 0.6, 0.6))
plt.title("Systemic crises for countries while being independent")
plt.show()

Data shows that systemic crisis started after counties became independent.

Central African Republic has the highest value for systemic crisis count. Angola, Mauritius and South Africa gave not experienced this crisis inn the given period.
And what about other crises?

# Banking crisis

A banking crisis is a financial crisis that affects banking activity. Banking crises include bank runs, which affect single banks; banking panics, which affect many banks; and systemic banking crises, in which a country experiences many defaults and financial institutions and corporations face great difficulties repaying contracts.A banking crisis is marked by bank runs that lead to the demise of financial institutions, or by the demise of a financial institution that starts a string of similar demises.


[Wikipedia](https://en.wikipedia.org/wiki/List_of_banking_crises)

In [ ]:

bank_crises_per_country_dependent = dependent.groupby("country",as_index=False)["banking_crises"].sum() 
bank_crises_per_country_independent = independent.groupby("country",as_index=False)["banking_crises"].sum() 
countrues_sorted_dep_bank = bank_crises_per_country_dependent.sort_values(by = ["banking_crises"], ascending=False)
countrues_sorted_indep_bank = bank_crises_per_country_independent.sort_values(by = ["banking_crises"], ascending=False)
plt.figure(figsize=(15,7))
plt.bar(countrues_sorted_dep_bank["country"], countrues_sorted_dep_bank["banking_crises"],color=(0.2, 0.2, 0.2, 0.6))
plt.title("Banking crises for countries while being dependent")
plt.show()
plt.figure(figsize=(15,7))
plt.bar(countrues_sorted_indep_bank["country"], countrues_sorted_indep_bank["banking_crises"],color=(0.9, 0.9, 0.6, 0.6))
plt.title("Banking crises for countries while being independent")
plt.show()

All countries between after became independent were affected by banking crisis. Central African Republic, Zimbabwe and Egypt suffered the most. 

It looks that there certainly is an interconnection between banking and systemic crisis. Let's take a look!

# Systemic and banking crisis

Here we examine if systematic crisis always means banking crisis too. Let's take al date regardless of being dependent or not.

In [ ]:
data_with_sys_crisis =  df[df.systemic_crisis == 1]
data_with_sys_crisis.country.unique()

First of all, collect all data where systemic crisis did happen...

In [ ]:
plt.figure(figsize=(10,6))
bank_sum_for_sys = data_with_sys_crisis.groupby("country",as_index=False)["banking_crises"].sum() 
bank_and_sys_countries_sorted = bank_sum_for_sys.sort_values(by = ["banking_crises"], ascending=False)
plt.figure(figsize=(20,6))
plt.bar(bank_and_sys_countries_sorted["country"], bank_and_sys_countries_sorted["banking_crises"],color=(0.2, 0.5, 0.9, 0.6))

And after that make a barchart with those countries who faced systemic crisis and show how often did they have banking crisis. 

The fact form the dataset is that **if systemic crisis happened, banking crisis will also affect the country**

Do they happen at the same time?

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.subplots_adjust(hspace=0.7, wspace=0.4)
for i in range(0, len(countries)):
    ax = fig.add_subplot(4, 4, i + 1)
    sns.barplot(x="year", y="systemic_crisis", data=df[df.country == countries[i]])
    plt.title(countries[i])

In [ ]:
fig = plt.figure(figsize=(15,10))
fig.subplots_adjust(hspace=0.7, wspace=0.4)
for i in range(0, len(countries)):
    ax = fig.add_subplot(4, 4, i + 1)
    sns.barplot(x="year", y="banking_crises", data=df[df.country == countries[i]])
    plt.title(countries[i])

Yep! Banking crisis definitely follow the time systemic risk happens. 

Finally, let's see heatmap od correlations:

In [ ]:
years = df.year.unique()
plt.figure(figsize=(10,10))
plt.title('Heatmap')
sns.heatmap(df.corr(), annot=True, cmap=sns.diverging_palette(20, 220, n=200))

It's clearly seen than banking crises is highly positively correlated with systemic crises.

# Inflation crisis and inflation rate

Inflation crisis means very high and typically accelerating inflation. It quickly erodes the real value of the local currency, as the prices of all goods increase. This causes people to minimize their holdings in that currency as they usually switch to more stable foreign currencies. There is no need to explain why this crisis is dangerous and dectructive for the economy. 

In [ ]:
inf_crises_per_country_dependent = dependent.groupby("country",as_index=False)["inflation_crises"].sum() 
inf_crises_per_country_independent = independent.groupby("country",as_index=False)["inflation_crises"].sum() 
countrues_sorted_dep_inf = inf_crises_per_country_dependent.sort_values(by = ["inflation_crises"], ascending=False)
countrues_sorted_indep_inf = inf_crises_per_country_independent.sort_values(by = ["inflation_crises"], ascending=False)
plt.figure(figsize=(15,7))
plt.bar(countrues_sorted_dep_inf["country"], countrues_sorted_dep_inf["inflation_crises"],color=(0.7, 0.7, 0.7, 0.6))
plt.title("Inflation crises for countries while being dependent")
plt.show()
plt.figure(figsize=(15,7))
plt.bar(countrues_sorted_indep_inf["country"], countrues_sorted_indep_inf["inflation_crises"],color=(0.3, 0.4, 0.6, 0.6))
plt.title("Inflation crises for countries while being independent")
plt.show()

Here situation has slightly changed - inflation crisis have already happened even before getting independent (in morocco, Algeria, Tunisia, Angola, Ivory coast and Mauritius).
In Marocco and Tunisia such crisis haven't repeated after but other countiries faced hyperinflation several times after that. 

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.subplots_adjust(hspace=0.7, wspace=0.4)
plt.style.use('seaborn-paper')
for i in range(0, len(countries)):
    ax = fig.add_subplot(4, 4, i + 1)
    df_country = df[df.country == countries[i]]
    sns.lineplot(df_country.year, df.inflation_annual_cpi, ci = None)
    plt.title(countries[i])

Here we cann idenntify when inflatio rate was the highest for each coutry. But what about independence? 

In [ ]:
sns.set_style("ticks", {"xtick.major.size": 10, "ytick.major.size": 10})
plt.style.use('seaborn-paper')
fig, axes = plt.subplots(ncols=4, nrows=4, figsize=(15,15))
axes = axes.flatten()
for i, ax in zip(countries, axes):
  sns.lineplot(x = 'year', y = 'inflation_annual_cpi', hue = 'independence', 
               data = df[df['country'] == i], ax = ax)
  ax.set_xlabel('Year')
  ax.set_ylabel('Inflation rate')
  ax.set_title('{}'.format(i))
  ax.get_legend().remove()
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc=1)
fig.subplots_adjust(top=0.95)
for i in range(13,16):
  fig.delaxes(axes[i])
plt.tight_layout()

Those graphics show that for some countries getting idependent helped to stabilize the iflation (Tunisia, Marocco) but for others it only caused new iflatio crises (Nigeria, Kenya, Agola, etc)

# Currency crisis 


A currency crisis is a situation in which serious doubt exists as to whether a country's central bank has sufficient foreign exchange reserves to maintain the country's fixed exchange rate. The crisis is often accompanied by a speculative attack in the foreign exchange market. A currency crisis results from chronic balance of payments deficits, and thus is also called a balance of payments crisis. Often such a crisis culminates in a devaluation of the currency.


[Wikipedia](https://en.wikipedia.org/wiki/Currency_crisis)

In [ ]:
curr_crises_per_country_dependent = dependent.groupby("country",as_index=False)["currency_crises"].sum() 
curr_crises_per_country_independent = independent.groupby("country",as_index=False)["currency_crises"].sum() 
countrues_sorted_dep_curr = curr_crises_per_country_dependent.sort_values(by = ["currency_crises"], ascending=False)
countrues_sorted_indep_curr = curr_crises_per_country_independent.sort_values(by = ["currency_crises"], ascending=False)
plt.figure(figsize=(15,7))
plt.bar(countrues_sorted_dep_curr["country"], countrues_sorted_dep_curr["currency_crises"],color=(0.1, 0.1, 0.1, 0.6))
plt.title("Currency crises for countries while being dependent")
plt.show()
plt.figure(figsize=(15,7))
plt.bar(countrues_sorted_indep_curr["country"], countrues_sorted_indep_curr["currency_crises"],color=(0.4, 0.7, 0.1, 0.6))
plt.title("Currency crises for countries while being independent")
plt.show()

# Exchange rate

Exchange rate is the rate at which one currency will be exchanged for another. It is also regarded as the value of one country's currency in relation to another currency
Exchange rate shows the state of country's economy and determines whether this country mostly focuses on import of export. In case of low exchange rate of national currency in country X, it's cheaper for other countries to import goods from country X. Consequently, if the exchange rate of national currency is high, country X is an importer and it's more affordable to buy goods from the foreign countries instead of producting them. 
If exchange rate fluctuates widely, it could create uncertainty and instability on financial markets, for trading processes and etc, and lead to critical situations for the whole country.

In [ ]:
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
plt.style.use('seaborn-paper')
fig, axes = plt.subplots(ncols=4, nrows=4, figsize=(15,15))
axes = axes.flatten()
for i, ax in zip(countries, axes):
  sns.lineplot(x = 'year', y = 'exch_usd', hue = 'independence', 
               data = df[df['country'] == i], ax = ax)
  ax.set_xlabel('Year')
  ax.set_ylabel('Exchange Rate')
  ax.set_title('{}'.format(i))
  ax.get_legend().remove()
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc=1)
fig.subplots_adjust(top=0.95)
for i in range(13,16):
  fig.delaxes(axes[i])
plt.tight_layout()

Exchange rate behavior is different for all countries, there is no united trend. However, for some countries exchange rate raised signiificantly or became unnstable after getting independent. In cintrast, in Tunisia is fell dramatically. 

# Results

1. Systemic crisis happened significantly more often after countries became independent
2. Banking crisis happened significantly more often after countries became independent
3. Currency crisis happened more often after countries became independent
4. Inflation crisis happened more often after countries became independent
3. If country faced systemic crisis, then it's probable that banking crisis will also happen shortly
4. For the majority of country presented independence led to the raise of inflation rate
5. Central African Rebulic is more prone to systemic and banking crises
6. Exchange rate after getting independent did't remain stable. it either raised significantly or fell dramatically

As a conclusion, independence is a huge step forward for the country, however, it leads to certain economic risks and even crises involving all sectors of the economy. 